In [1]:
import numpy as np

np.set_printoptions(precision=5,suppress=True,linewidth=np.inf)

import numpy as np

from dedalus_sphere.spin_operators import *

# helper functions numpy arrays

def iprint(M):
    print(M.astype(int))
    print()
    
def flat(*args): 
    for a in args:
        a.shape = (dim**2,)
        iprint(a)
        print()
    return None

def square(*args):
    for a in args:
        a.shape = (dim,dim)
        iprint(a)
        print()
    return None

def chop(M,eps=1e-9):
    
    w = np.where(np.abs(M) < eps)
    
    M[w] = 0
    
    print(M)
    print()


In regularity space operators are of the form:

$$\large
\mathrm{S}_{\ell}(\text{a},\text{b}) \ = \ \sum_{\sigma,\tau} \mathcal{Q}_{\ell}(\text{b},\sigma) S(\sigma,\tau) \mathcal{Q}_{\ell}(\text{a},\tau)
$$

$$\large
\mathrm{T}_{\ell}(\text{a},\text{b}) \ = \ \sum_{\sigma,\tau} \mathcal{Q}_{\ell}(\text{b},\sigma) T(\sigma,\tau) \mathcal{Q}_{\ell}(\text{a},\tau)
$$

The coeficient inbetween the $\mathcal{Q}$s are spin coupling coefficients. The $\ell$ dependence in regularity space makes everything much more annoying to work with. 

However, we can compose operators in spin space and then compute the regularity space action after the fact.

$$\large
(\mathrm{TS})_{\ell}(\text{a},\text{b}) \ = \ \sum_{\sigma,\tau} \mathcal{Q}_{\ell}(\text{b},\sigma) \left(\sum_{\kappa} T(\sigma,\kappa) S(\kappa,\tau) \right) \mathcal{Q}_{\ell}(\text{a},\tau)
$$

Being able to compose things allows us to build up any complicated thing in terms of elementary operations in spin space. 

As far as I can see the elementary operations are 

* Trace over any two tensor indices. 



* Transpose (or permutation) over any collection of indices.



* Tensor product with any tensorial spin basis element.



* Multiplication by the identity.



* Multiplication by the metric (physical space identity).



* Multiplication be the completly antisymmetric tensor of rank-3.




* The linear combination of compatible (same input and output) operators.



* The adjoint (transpose) of any of the above operators (reversing input and output). 



* The composition of any compatible operators.

In [28]:
spins  = (-1,0,+1)
dim    = len(spins)

I  = Identity(indexing=spins)

M  = Metric(indexing=spins)

T  = Transpose(permutation=(1,0),indexing=spins)

Tr = Trace((0,1),indexing=spins)

P  = lambda kappa: TensorProduct(kappa,indexing=spins)

Q = lambda ell: Intertwiner(ell,indexing=spins)

In [29]:
from operators import Codomain

In [30]:
c = Codomain(Tr.codomain,P((0,)).codomain)
d = Codomain(Tr.codomain,P((1,)).codomain)

In [31]:
c | d is c 

False

In [3]:
A = np.arange(dim**2,dtype=float)

iprint(A)

[0 1 2 3 4 5 6 7 8]



In [4]:
B = T(2) @ A

iprint(B)

square(A,B)

[0 3 6 1 4 7 2 5 8]

[[0 1 2]
 [3 4 5]
 [6 7 8]]


[[0 3 6]
 [1 4 7]
 [2 5 8]]




The rank-2 transpose operator matrix

In [5]:
iprint(T(2))

[[1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1]]



The trace in spin space is 

$$\large
\text{Tr}(A) = A(-1,+1) + A(0,0) + A(+1,-1)
$$

In [6]:
square(A)

if dim == 3: iprint(A[0,2] + A[2,0] + A[1,1])
if dim == 2: iprint(A[0,1] + A[1,0])

[[0 1 2]
 [3 4 5]
 [6 7 8]]


12



The metric moves the cross diagonal into the diagonal

In [7]:
iprint(M(1))
iprint(A)
iprint( M(1) @ A )
iprint(np.trace(M(1) @ A))

[[0 0 1]
 [0 1 0]
 [1 0 0]]

[[0 1 2]
 [3 4 5]
 [6 7 8]]

[[6 7 8]
 [3 4 5]
 [0 1 2]]

12



Re-flatten `A` and take the trace operator

In [8]:
flat(A)

iprint(Tr(2) @ A)

[0 1 2 3 4 5 6 7 8]


[12]



The rank-2 trace operator matrix

In [9]:
iprint(Tr(2))

[[0 0 1 0 1 0 1 0 0]]



The non-zeros spin elements:

In [10]:
print(Tr[(),(-1,+1)])
print(Tr[(),( 0, 0)])
print(Tr[(),(+1,-1)])

1
1
1


The trace is a left eigenvector of transpose.

In [11]:
iprint(Tr(2) @ T(2) - Tr(2))

[[0 0 0 0 0 0 0 0 0]]



This also works constructing the same thing as abstract operators.

In [12]:
Z = Tr @ T - Tr

iprint(Z(2))

[[0 0 0 0 0 0 0 0 0]]



We can construct the cotrace operator as the transpose.

In [13]:
Cotr = Tr.T

iprint(Cotr(0))

[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]]



The square operator multiplies by the dimension:

In [14]:
iprint((Tr @ Cotr)(0))
iprint((Tr @ Cotr)(1))
iprint((Tr @ Cotr)(2))

[[3]]

[[3 0 0]
 [0 3 0]
 [0 0 3]]

[[3 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 3]]



We can also use the trace to get the spin-0 element of a vector:

In [15]:
Trace(0)(1)

array([[0., 1., 0.]])

Same for a tensor:

In [16]:
(Trace(0) @ Trace(1))(2)

array([[0., 0., 0., 0., 1., 0., 0., 0., 0.]])

Or to project out the angular trace:

In [17]:
(Trace((0,1)) - Trace(0) @ Trace(1))(2)

array([[0., 0., 1., 0., 0., 0., 1., 0., 0.]])

This projects out the radial part of a vector:

In [18]:
(Trace(()) - (Trace(0).T) @ Trace(0))(1)

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.]])

BTW:

We can do some pretty crazy things:

In [19]:
(Trace((0,1,2)) -  Trace(0) @ ( Trace((0,1)) + Trace((0,2)) + Trace((1,2)) - 2 * Trace(0) @ Trace(0) ) ) (3)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

What about this?

$$\large
\text{S} \, \circ \, T \ = \ T + T^{\top} - \frac{2}{d}\, \text{Tr}(T)\, I 
$$

In [20]:
S = I + T - (2/I.dimension)*Cotr @ Tr

chop(S(2))

[[ 2.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       1.       0.       1.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.33333  0.      -0.66667  0.       0.33333  0.       0.     ]
 [ 0.       1.       0.       1.       0.       0.       0.       0.       0.     ]
 [ 0.       0.      -0.66667  0.       1.33333  0.      -0.66667  0.       0.     ]
 [ 0.       0.       0.       0.       0.       1.       0.       1.       0.     ]
 [ 0.       0.       0.33333  0.      -0.66667  0.       0.33333  0.       0.     ]
 [ 0.       0.       0.       0.       0.       1.       0.       1.       0.     ]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       2.     ]]



In [21]:
chop((Tr @ S)(2))

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]]



In [22]:
Tr2 = Cotr @ Tr 
iprint(Tr2(2))

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]



Evaluation keeps track of the rank

In [23]:
iprint((Cotr @ Tr)(2) - Cotr(0) @ Tr(2))

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]



Same with acting against arrays

What is the `Cotr @ Tr` operator?

Here is the Metric

In [24]:
iprint(M(1))
iprint(M(2))

[[0 0 1]
 [0 1 0]
 [1 0 0]]

[[0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]]



In [25]:
B = (Cotr @ Tr)(2) @ A
iprint(M(1)*(Tr(2) @ A))
iprint((Tr(2) @ A)*M(1))
square(B)

[[ 0  0 12]
 [ 0 12  0]
 [12  0  0]]

[[ 0  0 12]
 [ 0 12  0]
 [12  0  0]]

[[ 0  0 12]
 [ 0 12  0]
 [12  0  0]]




The `Cotr @ Tr` operator multiplies the trace times the metric identity.

We can interact with the spin elements directly:

In [26]:
print(T[(-1,+1),(+1,-1)])
print(Tr[(),(+1,-1)])
print(Tr[(),(+1,+1)])

1
1
0


The tensor product depends on the spin basis element.

Here is a spin e(1) acting on a rank-k field (scalar,vector,2-tensor):

$$\large
e(1) \otimes T \quad \text{as an abstract operator}
$$

In [27]:
iprint(P(+1)(0))
iprint(P(+1)(1))
iprint(P(+1)(2))

[[0]
 [0]
 [1]]

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]]

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]



We can linearly combine tensor products over basis elements:

In [28]:
S = 7*P(-1) + 3*P(0) - 2*P(1)

iprint(S(0))
iprint(S(1))
iprint(S(2))

[[ 7]
 [ 3]
 [-2]]

[[ 7  0  0]
 [ 0  7  0]
 [ 0  0  7]
 [ 3  0  0]
 [ 0  3  0]
 [ 0  0  3]
 [-2  0  0]
 [ 0 -2  0]
 [ 0  0 -2]]

[[ 7  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0]
 [ 0  0  0  0  0  0  7  0  0]
 [ 0  0  0  0  0  0  0  7  0]
 [ 0  0  0  0  0  0  0  0  7]
 [ 3  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0]
 [ 0  0  0  0  0  0  3  0  0]
 [ 0  0  0  0  0  0  0  3  0]
 [ 0  0  0  0  0  0  0  0  3]
 [-2  0  0  0  0  0  0  0  0]
 [ 0 -2  0  0  0  0  0  0  0]
 [ 0  0 -2  0  0  0  0  0  0]
 [ 0  0  0 -2  0  0  0  0  0]
 [ 0  0  0  0 -2  0  0  0  0]
 [ 0  0  0  0  0 -2  0  0  0]
 [ 0  0  0  0  0  0 -2  0  0]
 [ 0  0  0  0  0  0  0 -2  0]
 [ 0  0  0  0  0  0  0  0 -2]]



We can see that `S` increases the rank of anything by one.

In [29]:
S.codomain

(rank->rank+1)

For any input with rank > 0 we can contract over the (0,1) indices with `Tr`

$$\large
(7 e(-) + 3 e(0) - 2 e(+)) \cdot T
$$

In [30]:
(Tr @ S).codomain

(rank->rank-1)

In [31]:
iprint((Tr @ S)(1))
iprint((Tr @ S)(2))

[[-2  3  7]]

[[-2  0  0  3  0  0  7  0  0]
 [ 0 -2  0  0  3  0  0  7  0]
 [ 0  0 -2  0  0  3  0  0  7]]



For 2 < rank we can contract twice:

$$\large
\text{Tr} \left( (7 e(-) + 3 e(0) - 2 e(+)) \cdot T \right)
$$

In [32]:
print((Tr @ Tr @ S).codomain)
iprint((Tr @ Tr @ S)(3))

(rank->rank-3)
[[ 0  0 -2  0 -2  0 -2  0  0  0  0  3  0  3  0  3  0  0  0  0  7  0  7  0  7  0  0]]



In general we ca use different trace operators to contract over different indices:

In [33]:
Tr = lambda i,j: Trace((i,j),indexing=spins)

In [34]:
iprint((Tr(0,1) @ Tr(0,1) @ S)(3))
iprint((Tr(0,1) @ Tr(0,2) @ S)(3))
iprint((Tr(0,1) @ Tr(0,3) @ S)(3))
iprint((Tr(0,1) @ Tr(1,2) @ S)(3))
iprint((Tr(0,1) @ Tr(2,3) @ S)(3))
iprint((Tr(0,1) @ Tr(1,3) @ S)(3))

[[ 0  0 -2  0 -2  0 -2  0  0  0  0  3  0  3  0  3  0  0  0  0  7  0  7  0  7  0  0]]

[[ 0  0 -2  0  0  3  0  0  7  0 -2  0  0  3  0  0  7  0 -2  0  0  3  0  0  7  0  0]]

[[ 0  0  0  0  0  0 -2  3  7  0  0  0 -2  3  7  0  0  0 -2  3  7  0  0  0  0  0  0]]

[[ 0  0  0  0  0  0 -2  3  7  0  0  0 -2  3  7  0  0  0 -2  3  7  0  0  0  0  0  0]]

[[ 0  0 -2  0 -2  0 -2  0  0  0  0  3  0  3  0  3  0  0  0  0  7  0  7  0  7  0  0]]

[[ 0  0 -2  0  0  3  0  0  7  0 -2  0  0  3  0  0  7  0 -2  0  0  3  0  0  7  0  0]]



Thinking of $\mathrm{V}$ as the tensor-product operator, and $T$ as the rank-3 tensor input, these matrices represent the following respective actions:

$$\large
\sum_{\sigma,\tau} \mathrm{V}_{\sigma}\ T_{-\sigma,\,\tau,\,-\tau}, 
\quad \sum_{\sigma,\tau} \mathrm{V}_{\sigma}\ T_{\tau,\,-\sigma,\,-\tau}, \quad
\sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{\sigma,\,-\sigma,\,-\tau}
$$


$$\large
\sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{\sigma,\,-\sigma,\,-\tau}, \quad \sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{\sigma,\,-\tau,\,-\sigma},\quad
\sum_{\sigma,\tau} \mathrm{V}_{\tau}\ T_{-\tau\,\,\sigma,\,-\sigma}
$$

Because we can reverse the order of the sums:

In [35]:
rank = 3

Z1 = Tr(0,1) @ (Tr(0,1)-Tr(2,3)) @ S
Z2 = Tr(0,1) @ (Tr(0,2)-Tr(1,3)) @ S
Z3 = Tr(0,1) @ (Tr(0,3)-Tr(1,2)) @ S
iprint(Z1(rank))
iprint(Z2(rank))
iprint(Z3(rank))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]



For rank-4;

In [36]:
rank = 4
iprint(Z1(rank))
iprint(Z2(rank))
iprint(Z3(rank))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]

[[0 0 

Last we have the spin-to-regularity intertwiners.

For a sanity check, we get the right answer compared to the old version.

In [37]:
from old_intertwiner import regularity2spinMap as old_Q

max_rank = 3
max_ell  = 10

good = True
for r in range(max_rank+1):
    for l in range(max_ell+1):
        for i in Q(l).range(2*r):
            good = good and abs(old_Q(l,i[:r],i[r:]) - Q(l)[i[:r],i[r:]]) < Q(l).threshold
print(good)

True


For rank=1:

In [38]:
for ell in range(5):
    chop(Q(ell)(1))
    print()

[[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 0.]]


[[ 0.57735  0.70711 -0.40825]
 [ 0.57735  0.       0.8165 ]
 [-0.57735  0.70711  0.40825]]


[[ 0.54772  0.70711 -0.44721]
 [ 0.63246  0.       0.7746 ]
 [-0.54772  0.70711  0.44721]]


[[ 0.53452  0.70711 -0.46291]
 [ 0.65465  0.       0.75593]
 [-0.53452  0.70711  0.46291]]


[[ 0.52705  0.70711 -0.4714 ]
 [ 0.66667  0.       0.74536]
 [-0.52705  0.70711  0.4714 ]]




Calling on rank=2:

In [39]:
for ell in range(4):
    chop(Q(ell)(2))
    print()

[[ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.57735  0.       0.       0.70711  0.       0.      -0.40825]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.57735  0.       0.       0.       0.       0.       0.8165 ]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.57735  0.       0.      -0.70711  0.       0.      -0.40825]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]]


[[ 0.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.      -0.40825  0.3873   0.      -0.5      0.5      0.       0.28868 -0.31623]
 [ 0.       0.40825  0.3873   0.       0.5      0.5      0.      -0.28868

We can also index the elements directly:

In [40]:
spin = 0
regularity = 1

Q(0)[spin,regularity]


spin = (-1,1)
regularity = (0,1)

Q(0)[spin,regularity]

0.7071067811865475

We can see the operator is orthogonal

In [41]:
for ell in range(4):

    chop((Q(ell).T @ Q(ell))(1))

    chop((Q(ell) @ Q(ell).T)(1))

    print()

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]




In [42]:
for ell in range(3):

    chop((Q(ell).T @ Q(ell))(2))

    chop((Q(ell) @ Q(ell).T)(2))

    print()


[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0

We can build regularity-space operators out of the spin representations

For the metric:

In [43]:
for ell in range(4):
    chop((Q(ell).T @ M @ Q(ell))(1))

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]]

[[-0.33333  0.       0.94281]
 [ 0.       1.       0.     ]
 [ 0.94281  0.       0.33333]]

[[-0.2     0.      0.9798]
 [ 0.      1.      0.    ]
 [ 0.9798  0.      0.2   ]]

[[-0.14286  0.       0.98974]
 [ 0.       1.       0.     ]
 [ 0.98974  0.       0.14286]]



For the trace:

In [44]:
TRACE = np.zeros((3,3))
for ell in range(4):
    for i,a in enumerate((-1,0,1)):
        for j,b in enumerate((-1,0,1)):
            TRACE[i,j] = (Q(ell).T @ Tr(0,1) @ Q(ell))[(),(a,b)]
    chop(TRACE)

[[0.      0.      1.73205]
 [0.      0.      0.     ]
 [0.      0.      0.     ]]

[[0.      0.      1.29099]
 [0.      1.      0.     ]
 [0.57735 0.      0.     ]]

[[0.      0.      1.18322]
 [0.      1.      0.     ]
 [0.7746  0.      0.     ]]

[[0.      0.      1.13389]
 [0.      1.      0.     ]
 [0.84515 0.      0.     ]]



This should be:
$$
\left[
\begin{array}{ccc}
 0 & 0 & \frac{\sqrt{2 \ell +3}}{\sqrt{2 \ell +1}} \\
 0 & 1 & 0 \\
 \frac{\sqrt{2 \ell -1}}{\sqrt{2 \ell +1}} & 0 & 0 \\
\end{array}
\right]
$$

In [45]:
for ell in range(1,4):
    print(np.sqrt((2*ell-1)/(2*ell+1)),1,np.sqrt((2*ell+3)/(2*ell+1)))

0.5773502691896257 1 1.2909944487358056
0.7745966692414834 1 1.1832159566199232
0.8451542547285166 1 1.1338934190276817


For the transpose:

In [46]:
ell = 3
chop((Q(ell).T @ T @ Q(ell))(2))

[[ 1.       0.       0.       0.       0.       0.       0.       0.       0.     ]
 [ 0.      -0.33333  0.       0.94281  0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.03571  0.       0.28347  0.       0.95831  0.       0.     ]
 [ 0.       0.94281  0.       0.33333  0.       0.       0.       0.       0.     ]
 [ 0.       0.       0.28347  0.       0.91667  0.      -0.28172  0.       0.     ]
 [ 0.       0.       0.       0.       0.      -0.25     0.       0.96825  0.     ]
 [ 0.       0.       0.95831  0.      -0.28172  0.       0.04762  0.       0.     ]
 [ 0.       0.       0.       0.       0.       0.96825  0.       0.25     0.     ]
 [ 0.       0.       0.       0.       0.       0.       0.       0.       1.     ]]



In [47]:
reg0 = ((-1,+1),(0,0),(+1,-1))

DAG = np.zeros((3,3))
for ell in range(4):
    for i,a in enumerate(reg0):
        for j,b in enumerate(reg0):
            DAG[i,j] = (Q(ell).T @ T @ Q(ell))[a,b]
    chop(DAG)

[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

[[ 0.16667  0.6455   0.74536]
 [ 0.6455   0.5     -0.57735]
 [ 0.74536 -0.57735  0.33333]]

[[ 0.06667  0.39441  0.91652]
 [ 0.39441  0.83333 -0.3873 ]
 [ 0.91652 -0.3873   0.1    ]]

[[ 0.03571  0.28347  0.95831]
 [ 0.28347  0.91667 -0.28172]
 [ 0.95831 -0.28172  0.04762]]



This should be:

$$
\left[
\begin{array}{ccc}
 \frac{1}{(\ell +1) (2 \ell +1)} & \frac{\sqrt{2 \ell
   +3}}{(\ell +1) \sqrt{2 \ell +1}} & \frac{\sqrt{(2
   \ell -1) (2 \ell +3)}}{2 \ell +1} \\
 \frac{\sqrt{2 \ell +3}}{(\ell +1) \sqrt{2 \ell +1}} &
   1-\frac{1}{\ell  (\ell +1)} & -\frac{\sqrt{2 \ell
   -1}}{\ell  \sqrt{2 \ell +1}} \\
 \frac{\sqrt{(2 \ell -1) (2 \ell +3)}}{2 \ell +1} &
   -\frac{\sqrt{2 \ell -1}}{\ell  \sqrt{2 \ell +1}} &
   \frac{1}{\ell  (2 \ell +1)} \\
\end{array}
\right]
$$


<br>

Here's how we can build up NCC operators in regularity space. 

For one example:

In [48]:
NCC = lambda l, a: Q(l).T @ sum( Q(0)[e,a] * Trace((len(e)-1,len(e))) @ P(e) for e in Q(0).range(len(a))) @ Q(l)

Here are the three non-zero regularity coupling matrices that compute matrix multiplication operators

In [49]:
ell = 10

for a in ((-1,1),(0,1),(1,1)):
    print(NCC(ell,a)(1))
    print()

[[0.57735 0.      0.     ]
 [0.      0.57735 0.     ]
 [0.      0.      0.57735]]

[[ 0.       0.51177  0.     ]
 [ 0.51177  0.      -0.48795]
 [ 0.      -0.48795  0.     ]]

[[ 0.17496  0.       0.61168]
 [ 0.      -0.40825  0.     ]
 [ 0.61168  0.       0.23328]]



The others vanish

In [50]:
for s in (-1,0,1):
    print(NCC(ell,(s,-1))(1))
    print()
    print(NCC(ell,(s,0))(1))
    print()

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]



We can check the selection rules:

In [51]:
for rank in (1,2,3):
    
    print()
    print()
    print('rank:',rank)

    print(' _   _   _     _     _    _  ')
    print(' a   b   c  :  a - | b -  c |')
    print('--- --- ---   ----------------')
    for a in Q(0).range(rank):
        if not Q(0).forbidden_regularity(a):
            for b in NCC(ell,a).range(rank-1):
                for c in NCC(ell,a).range(1):
                    if NCC(ell,a)[b,c] != 0:
                        rule = sum(a) - abs(sum(b)-sum(c))
                        print(f'{sum(a):>2}, {sum(b):>2}, {sum(c):>2}  :  {rule}')



rank: 1
 _   _   _     _     _    _  
 a   b   c  :  a - | b -  c |
--- --- ---   ----------------
 1,  0, -1  :  0
 1,  0,  1  :  0


rank: 2
 _   _   _     _     _    _  
 a   b   c  :  a - | b -  c |
--- --- ---   ----------------
 0, -1, -1  :  0
 0,  0,  0  :  0
 0,  1,  1  :  0
 1, -1,  0  :  0
 1,  0, -1  :  0
 1,  0,  1  :  0
 1,  1,  0  :  0
 2, -1, -1  :  2
 2, -1,  1  :  0
 2,  0,  0  :  2
 2,  1, -1  :  0
 2,  1,  1  :  2


rank: 3
 _   _   _     _     _    _  
 a   b   c  :  a - | b -  c |
--- --- ---   ----------------
 0, -1, -1  :  0
 0,  0,  0  :  0
 0, -1, -1  :  0
 0,  0,  0  :  0
 0,  1,  1  :  0
 0,  0,  0  :  0
 0,  1,  1  :  0
 1, -2, -1  :  0
 1, -1,  0  :  0
 1,  0, -1  :  0
 1,  0,  1  :  0
 1, -1,  0  :  0
 1,  0, -1  :  0
 1,  0,  1  :  0
 1,  1,  0  :  0
 1,  0, -1  :  0
 1,  0,  1  :  0
 1,  1,  0  :  0
 1,  2,  1  :  0
 1, -2, -1  :  0
 1, -1,  0  :  0
 1,  0,  1  :  0
 1, -1,  0  :  0
 1,  0, -1  :  0
 1,  0,  1  :  0
 1,  1,  0  :  0
 1,  0, -1  :  0
